In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
from PIL import Image 
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataDir='C:\\Users\\Sushanth\\Downloads\\CATS_DOGS\\'
with Image.open(dataDir+'test\\'+'CAT\\10107.jpg') as Im:
    display(Im)

In [ ]:
img_names=[]
for folder,subfolder,fnames in os.walk(dataDir):
    for img in fnames:
        img_names.append(folder+'\\'+img)


In [ ]:
img_sizes=[]
rejected=[]
for im_name in img_names:
    try:
        with Image.open(im_name) as img:
            img_sizes.append(img.size)
    except:
        rejected.append(im_name)

In [ ]:
df=pd.DataFrame(img_sizes)
df[0].describe()
df[1].describe()

In [ ]:
d1=Image.open(dataDir+'train\\Dog\\'+'14.jpg')
display(d1)

In [2]:
train_transform=torchvision.transforms.Compose([torchvision.transforms.RandomRotation(10),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

test_transform=torchvision.transforms.Compose([torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

In [3]:
root='C:\\Users\\Sushanth\\Downloads\\CATS_DOGS\\'
train_data=torchvision.datasets.ImageFolder(os.path.join(root,'train'),transform=train_transform)
test_data=torchvision.datasets.ImageFolder(os.path.join(root,'test'),transform=test_transform)
torch.manual_seed(101)
train_loader=torch.utils.data.DataLoader(train_data,batch_size=64,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=64)
class_names=train_data.classes

In [ ]:
for imgs,labels in train_loader:
    break

In [ ]:
Im=torchvision.utils.make_grid(imgs,nrow=5)
inv_normalize=torchvision.transforms.Normalize([-0.485/0.229,-0.456/0.229,-0.406/0.229],[1/0.229,1/0.224,1/0.225])
im_inv=inv_normalize(Im)
plt.figure(figsize=(10,10))
plt.imshow(np.transpose(im_inv.numpy(),[1,2,0]))

In [25]:
class convnet_cat_dog(nn.Module):
    def __init__(self):
        super(convnet_cat_dog,self).__init__()
        self.conv1=nn.Conv2d(3,6,3,1)
        self.conv2=nn.Conv2d(6,16,3,1)
        self.conv3=nn.Conv2d(16,32,3,1)
        self.conv4=nn.Conv2d(32,64,3,1)
        self.fc1=nn.Linear(64*12*12,2048)
        self.fc2=nn.Linear(2048,512)
        self.fc3=nn.Linear(512,64)
        self.fc4=nn.Linear(64,16)
        self.fc5=nn.Linear(16,2)
    def forward(self,X):
        C1=F.relu(self.conv1(X))
        p1=F.max_pool2d(C1,(2,2))
        C2=F.relu(self.conv2(p1))
        p2=F.max_pool2d(C2,(2,2))
        C3=F.relu(self.conv3(p2))
        p3=F.max_pool2d(C3,(2,2))
        C4=F.relu(self.conv4(p3))
        p4=F.max_pool2d(C4,(2,2))
        p4f=p4.view(-1,64*12*12)
        fc1_op=F.relu(self.fc1(p4f))
        fc2_op=F.relu(self.fc2(fc1_op))
        fc3_op=F.relu(self.fc3(fc2_op))
        fc4_op=F.relu(self.fc4(fc3_op))
        fc5_op=self.fc5(fc4_op)
        return(fc5_op)       

In [26]:
torch.manual_seed(42)
model=convnet_cat_dog()
crit=nn.CrossEntropyLoss()
opt=torch.optim.Adam(model.parameters())

In [27]:
def train_single_epoch(d_loader,mod,crit,opt,disp_int=30):
    running_batch_loss=np.zeros([len(d_loader),1])
    for batch,ex in enumerate(d_loader):
        opt.zero_grad()
        preds=mod.forward(ex[0])
        loss=crit(preds,ex[1])
        running_batch_loss[batch]=loss.item()
        loss.backward()
        opt.step()
        if(batch%disp_int==0):
            print(f'Batch:{batch} Loss:{loss.item():0.4f}')
    return(mod,running_batch_loss)
def train_model(epochs,d_loader,mod,crit,opt,disp_int=30):
    running_epoch_loss=[]
    for i in range(epochs):
        print(f'__________________Epoch: {i}_________________________________')
        mod,running_batch_loss=train_single_epoch(d_loader,mod,crit,opt,disp_int=disp_int)
        running_epoch_loss.extend(running_batch_loss)
    return(mod,running_epoch_loss)

epochs=10
mod,running_epoch_loss=train_model(epochs,train_loader,model,crit,opt,disp_int=30)

__________________Epoch: 0_________________________________
Batch:0 Loss:0.6944
Batch:30 Loss:0.6903
Batch:60 Loss:0.6923
Batch:90 Loss:0.6908
Batch:120 Loss:0.6694
Batch:150 Loss:0.6316
Batch:180 Loss:0.5990
Batch:210 Loss:0.5716
Batch:240 Loss:0.5856
Batch:270 Loss:0.6389
__________________Epoch: 1_________________________________
Batch:0 Loss:0.5296
Batch:30 Loss:0.5208
Batch:60 Loss:0.6304
Batch:90 Loss:0.5152
Batch:120 Loss:0.5277
Batch:150 Loss:0.5496
Batch:180 Loss:0.4783
Batch:210 Loss:0.5522
Batch:240 Loss:0.5959
Batch:270 Loss:0.4726
__________________Epoch: 2_________________________________
Batch:0 Loss:0.4647
Batch:30 Loss:0.5158
Batch:60 Loss:0.4506
Batch:90 Loss:0.4808
Batch:120 Loss:0.4358
Batch:150 Loss:0.4363
Batch:180 Loss:0.5585
Batch:210 Loss:0.5022
Batch:240 Loss:0.4202
Batch:270 Loss:0.4935
__________________Epoch: 3_________________________________
Batch:0 Loss:0.4437
Batch:30 Loss:0.4644
Batch:60 Loss:0.5619
Batch:90 Loss:0.5400
Batch:120 Loss:0.3637
Batch:150 

In [29]:
def eval_model(mod,d_loader):
    pred_probs=[]
    pred_labels=[]
    true_labels=[]
    for b,ex in enumerate(d_loader):
        preds=mod.forward(ex[0])
        prob=torch.softmax(preds,1)
        probs=np.max(prob.detach().numpy(),axis=1)
        labs=np.argmax(prob.detach().numpy(),axis=1)
        pred_probs.extend(probs)
        pred_labels.extend(labs)
        true_labels.extend(ex[1].detach().numpy())
    return(pred_probs,pred_labels,true_labels)



In [30]:
def get_conf_mat(pred_labels,true_labels,num_classes,disp_flag=0):
    cm=np.zeros([num_classes,num_classes])
    k=0
    for k in range(len(pred_labels)):
        cm[pred_labels[k],true_labels[k]]+=1
        k+=1
    if(disp_flag==1):
        print(cm)
    return(cm)


In [31]:
pred_probs_train,pred_labels_train,true_labels_train=eval_model(mod,train_loader)
pred_probs_test,pred_labels_test,true_labels_test=eval_model(mod,test_loader)

In [32]:
cm_train=get_conf_mat(pred_labels_train,true_labels_train,num_classes=2)

In [33]:
cm_test=get_conf_mat(pred_labels_test,true_labels_test,num_classes=2)

In [34]:
np.sum(np.diag(cm_train))/np.sum(cm_train)

0.9063650429493678

In [35]:
np.sum(np.diag(cm_test))/np.sum(cm_test)

0.8597024476083827